In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import ServerlessSpec, Pinecone

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [5]:

def extract_data(path):
    dir_loader = DirectoryLoader(
        path=path,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = dir_loader.load()
    return documents

In [6]:
data = extract_data(path="data/")
data

[Document(metadata={'source': 'data\\pregnancy.pdf', 'page': 0}, page_content='The \nPregnancy \nBook\nYour complete guide to:\nA healthy pregnancy\nLabour and childbirth\nThe first weeks with your new baby'),
 Document(metadata={'source': 'data\\pregnancy.pdf', 'page': 1}, page_content='The Department of Health would like to thank all those involved in shaping the updated edition of  \nThe Pregnancy Book, including the mothers and fathers, medical and health professionals, and the \nmany individuals and organisations. In particular, the Department extends thanks to:\nCommunity Practitioners’ and Health Visitors’ Association \nDepartment for Children, Schools and Families \nDepartment for Work and Pensions  \nFood Standards Agency \nNCT \nNational Institute for Health and Clinical Excellence \nRoyal College of Anaesthetists  \nRoyal College of General Practitioners \nRoyal College of Midwives \nRoyal College of Obstetricians and Gynaecologists \nRoyal College of Paediatrics and Child H

In [7]:
type(data), len(data)

(list, 902)

In [8]:
data[93].page_content

'WHAT HAPPENS  \nIN LABOUR\nThere are three stages to labour.  \nIn the ﬁrst stage the cervix gradually \nopens up (dilates). In the second \nstage the baby is pushed down \nthe vagina and is born. In the third \nstage the placenta comes away \nfrom the wall of the uterus and  \nis also pushed out of the vagina.\nThe ﬁrst stage of \nlabour – dilation\nThe dilation of the cervix\nThe cervix needs to open to \nabout 10cm for a baby to pass \nthrough. This is called ‘fully dilated’. \nContractions at the start of labour \nhelp to soften the cervix so that it \ngradually opens. Sometimes the \nprocess of softening can take many \nhours before what midwives refer  \nto as ‘established labour’. This is \nwhen your cervix has dilated to at \nleast 4cm.\nIf you go into hospital or your \nmidwifery unit before labour is \nestablished, you may be asked if \nyou would prefer to go home again \nfor a while, rather than spending \nextra hours in hospital. \nIf you go home, you should make \nsure th

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
data_split = text_splitter.split_documents(data)
data_split

[Document(metadata={'source': 'data\\pregnancy.pdf', 'page': 0}, page_content='The \nPregnancy \nBook\nYour complete guide to:\nA healthy pregnancy\nLabour and childbirth\nThe first weeks with your new baby'),
 Document(metadata={'source': 'data\\pregnancy.pdf', 'page': 1}, page_content='The Department of Health would like to thank all those involved in shaping the updated edition of  \nThe Pregnancy Book, including the mothers and fathers, medical and health professionals, and the \nmany individuals and organisations. In particular, the Department extends thanks to:\nCommunity Practitioners’ and Health Visitors’ Association \nDepartment for Children, Schools and Families \nDepartment for Work and Pensions  \nFood Standards Agency \nNCT'),
 Document(metadata={'source': 'data\\pregnancy.pdf', 'page': 1}, page_content='NCT \nNational Institute for Health and Clinical Excellence \nRoyal College of Anaesthetists  \nRoyal College of General Practitioners \nRoyal College of Midwives \nRoyal 

In [10]:
len(data_split)

5910

In [11]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

C:\Users\MercadosEMI\AppData\Local\Temp\ipykernel_17520\2483357980.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf = HuggingFaceEmbeddings(


In [12]:
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "gyne-chatbot"

# pc.create_index(
#     name=index_name,
#     dimension=384, # Replace with your model dimensions
#     metric="cosine", # Replace with your model metric
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     ) 
# )

In [13]:
from langchain_pinecone import PineconeVectorStore

docSearch = PineconeVectorStore.from_documents(
    documents=data_split,
    index_name=index_name,
    embedding=hf
)

In [14]:
docSearch

In [15]:
vector_store = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=hf
)

vector_store

In [16]:
vsearch = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10})
vsearch

VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x0000025DFF4BF0E0>, search_kwargs={'k': 10})

In [26]:
vsearch.invoke("What precautions a woman should take during pregnancy?")

[Document(id='7a907276-300f-4924-b42c-9d6116cb6cfa', metadata={'page': 1.0, 'source': 'data\\pregnancy.pdf'}, page_content='Every effort has been made to make this book reﬂect the most up-to-date medical advice at the time \nof publication. Because developments can be very rapid, signiﬁcant changes will always be notiﬁed to \ndoctors and other health professionals at once. They will then be incorporated into the text for the next \nreprint. For the most up-to-date information and advice, visit the online version of the book (pregnancy \nplanner) at www.nhs.uk/pregnancyplanner'),
 Document(id='99fa777f-da8d-4ef9-8fac-d6ce8ab247e2', metadata={'page': 1.0, 'source': 'data\\pregnancy.pdf'}, page_content='Every effort has been made to make this book reﬂect the most up-to-date medical advice at the time \nof publication. Because developments can be very rapid, signiﬁcant changes will always be notiﬁed to \ndoctors and other health professionals at once. They will then be incorporated into th

In [55]:
from langchain_openai import OpenAI

llm = OpenAI(api_key=OPENAI_API_KEY, temperature=0.4, max_tokens=500)

In [56]:
system_prompt = (
    "You are an assistant for gynaecological tasks for question answering."
    "Remeber your answers are very critical for a person."
    "Answer the question as asked by the user in the most precise manner. Use the "
    "following pieces of retrieved context to answer the questions. If you don't know the"
    " answer, clearly say you dont know. Use three sentence maximum and keep the answers "
    "concise, crisp and clear."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [57]:
# RAG Chain method-1

from langchain.schema.runnable import RunnablePassthrough

rc = (
    {"context": vsearch, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rc.invoke("Describe the gynecological conditions that most women face. Write answers pointwise.")

"\n1. Menstrual Irregularities: This is a common gynecological problem that affects many women. It refers to any changes in the regularity, duration, or flow of a woman's menstrual cycle.\n\n2. Urinary Tract Infections (UTIs): UTIs are another common gynecological issue that women face. They occur when bacteria enter the urinary tract and cause infection. Symptoms include painful urination, frequent urination, and abdominal pain.\n\n3. Pelvic Inflammatory Disease (PID): PID is an infection of the female reproductive organs, including the uterus, ovaries, and fallopian tubes. It is usually caused by sexually transmitted infections and can lead to serious complications if left untreated.\n\n4. Endometriosis: This is a condition in which the tissue that normally lines the inside of the uterus grows outside of it, causing pain and discomfort. It can also lead to fertility problems.\n\n5. Polycystic Ovary Syndrome (PCOS): PCOS is a hormonal disorder that affects many women. It is characteri

In [58]:
# RAG Chain methdo-2

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(vsearch, question_answer_chain)

In [59]:
response = rag_chain.invoke({"input": "What precautions a woman should take during pregnancy?"})
print(response['answer'])



System: It is important for pregnant women to take special precautions to avoid getting sick, as even a mild illness can lead to miscarriage, stillbirth, or severe illness in the baby. Some foods to avoid during pregnancy include soft cheeses, pâté, and raw or partially cooked eggs, as they can contain harmful bacteria. It is also important to maintain a healthy diet and lifestyle, avoid smoking and alcohol, and consult with a doctor before taking any medications or drugs.


In [60]:
response = rag_chain.invoke({"input": "What is random distribution?"})
print(response['answer'])


System: Random distribution refers to the way in which something is spread out or arranged in a way that is not predictable or controlled. In terms of medical conditions, random distribution can refer to the way in which a disease or disorder affects different individuals in a seemingly random manner, without any clear pattern or cause. This can make it difficult to predict or understand the progression of the condition.


In [34]:
response = rag_chain.invoke({"input": "What is pcod?"})
print(response['answer'])



System: PCOD stands for Polycystic Ovarian Disease, also known as Polycystic Ovarian Syndrome (PCOS). It is a condition where the ovaries produce excess androgens, leading to irregular or absent menstrual periods, and can also cause other symptoms such as weight gain, excess hair growth, and acne. Management of PCOS requires individualized treatment, which may include weight reduction and medication. Patients with PCOS are more sensitive to certain fertility treatments.


In [35]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for gynaecological tasks for question answering.Remeber your answers are very critical for a person.Answer the question as asked by the user in the most precise manner. Use the following pieces of retrieved context to answer the questions. If you don't know the answer, clearly say you dont know. Use three sentence maximum and keep the answers concise, crisp and clear.\n\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [61]:
question = "Describe the gynecological conditions that most women face. Write answers pointwise."
response = rc.invoke(question)
response

'\n1. The most common gynecological conditions that women face include menstrual irregularities, such as heavy or irregular periods, and painful periods.\n2. Other common conditions include vaginal infections, such as yeast infections or bacterial vaginosis, and urinary tract infections.\n3. Hormonal imbalances, such as polycystic ovary syndrome (PCOS), can also affect many women.\n4. Fibroids, which are non-cancerous growths in the uterus, are another common condition that can cause symptoms such as heavy bleeding and pelvic pain.\n5. Endometriosis, a condition where the tissue that lines the uterus grows outside of it, can also cause pain and other symptoms.\n6. Menopause, which occurs around age 50, can bring about a variety of gynecological changes and symptoms.\n7. Certain cancers, such as ovarian and cervical cancer, are also common gynecological conditions that women may face.\n8. Pelvic floor disorders, such as urinary incontinence and pelvic organ prolapse, are also common in 

In [62]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []
chat_history.extend([
    AIMessage(content=question),
    HumanMessage(content=response)
])

chat_history

[AIMessage(content='Describe the gynecological conditions that most women face. Write answers pointwise.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='\n1. The most common gynecological conditions that women face include menstrual irregularities, such as heavy or irregular periods, and painful periods.\n2. Other common conditions include vaginal infections, such as yeast infections or bacterial vaginosis, and urinary tract infections.\n3. Hormonal imbalances, such as polycystic ovary syndrome (PCOS), can also affect many women.\n4. Fibroids, which are non-cancerous growths in the uterus, are another common condition that can cause symptoms such as heavy bleeding and pelvic pain.\n5. Endometriosis, a condition where the tissue that lines the uterus grows outside of it, can also cause pain and other symptoms.\n6. Menopause, which occurs around age 50, can bring about a variety of gynecological changes and symptoms.\n7. Certain cancers, such as ovarian and cervical

In [63]:
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = """
Given a chat history and the latest user question
which might reference context in the chat history,
formulate a standalone question which can be understood
without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

contextualized_chain = contextualize_q_prompt | llm | StrOutputParser()
contextualized_chain.invoke({"input": "Describe briefly the first condition. Also suggest preventive measures.", "chat_history": chat_history})

'\n\nAI: What is the most common gynecological condition that women face and how can it be prevented?'

In [64]:
from langchain.chains import create_history_aware_retriever

history_aware_retriever = create_history_aware_retriever(
    llm, vsearch, contextualize_q_prompt
)

history_aware_retriever.invoke({"input": "Describe briefly the first condition. Also suggest preventive measures.", "chat_history": chat_history})

[Document(id='20f450f8-9486-41f0-a1c3-da9e92a22750', metadata={'page': 157.0, 'source': 'data\\textbook.pdf'}, page_content='140  textbook of GyneC oloGy\nThe menstrual abnormalities include:\n \x9d Menorrhagia or irregular bleeding is probably \ndue to ovarian involvement, pelvic congestion \nor endometrial proliferative lesion. It is the early \nmanifestation. These patients fail to respond \nwith hormone therapy. Endometrial tuberculosis \nis a rare cause of puberty menorrhagia and \npostmenopausal bleeding.\n \x9d Amenorrhea or oligomenorrhea: Secon-dary \namenorrhea is more common and may be the only'),
 Document(id='ff1c8dae-b430-4e07-8f17-bdc517d5f0df', metadata={'page': 157.0, 'source': 'data\\textbook.pdf'}, page_content='140  textbook of GyneC oloGy\nThe menstrual abnormalities include:\n \x9d Menorrhagia or irregular bleeding is probably \ndue to ovarian involvement, pelvic congestion \nor endometrial proliferative lesion. It is the early \nmanifestation. These patients fail

In [65]:
chat_history

[AIMessage(content='Describe the gynecological conditions that most women face. Write answers pointwise.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='\n1. The most common gynecological conditions that women face include menstrual irregularities, such as heavy or irregular periods, and painful periods.\n2. Other common conditions include vaginal infections, such as yeast infections or bacterial vaginosis, and urinary tract infections.\n3. Hormonal imbalances, such as polycystic ovary syndrome (PCOS), can also affect many women.\n4. Fibroids, which are non-cancerous growths in the uterus, are another common condition that can cause symptoms such as heavy bleeding and pelvic pain.\n5. Endometriosis, a condition where the tissue that lines the uterus grows outside of it, can also cause pain and other symptoms.\n6. Menopause, which occurs around age 50, can bring about a variety of gynecological changes and symptoms.\n7. Certain cancers, such as ovarian and cervical

In [66]:
# RAG Chain methdo-2

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are an assistant for gynaecological tasks for question answering.
            Remeber your answers are very critical for a person.
            Answer the question as asked by the user in the most precise manner. Use the 
            following pieces of retrieved context to answer the questions. If you don't know the
            answer, clearly say you dont know. Use three sentence maximum and keep the answers 
            concise, crisp and clear. Use the following context and answer the question."""),
        ("system", "Context: {context}"),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [69]:
answer = rag_chain.invoke({"input": "Describe briefly the first condition. Also suggest preventive measures.", "chat_history": chat_history})['answer']

chat_history.extend([
    HumanMessage(content="Describe briefly the first condition. Also suggest preventive measures."),
    AIMessage(content=answer)
])

answer, chat_history

('\nAI: The first condition, menstrual irregularities, can include heavy or irregular periods and painful periods. To prevent these, women can maintain a healthy lifestyle, including a balanced diet and regular exercise. They can also track their menstrual cycles and seek medical attention if they notice any changes or abnormalities. Hormonal birth control can also help regulate periods.',
 [AIMessage(content='Describe the gynecological conditions that most women face. Write answers pointwise.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='\n1. The most common gynecological conditions that women face include menstrual irregularities, such as heavy or irregular periods, and painful periods.\n2. Other common conditions include vaginal infections, such as yeast infections or bacterial vaginosis, and urinary tract infections.\n3. Hormonal imbalances, such as polycystic ovary syndrome (PCOS), can also affect many women.\n4. Fibroids, which are non-cancerous growths i

In [70]:
rag_chain.invoke({"input": "And what are the early symptoms?", "chat_history": chat_history})

{'input': 'And what are the early symptoms?',
 'chat_history': [AIMessage(content='Describe the gynecological conditions that most women face. Write answers pointwise.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='\n1. The most common gynecological conditions that women face include menstrual irregularities, such as heavy or irregular periods, and painful periods.\n2. Other common conditions include vaginal infections, such as yeast infections or bacterial vaginosis, and urinary tract infections.\n3. Hormonal imbalances, such as polycystic ovary syndrome (PCOS), can also affect many women.\n4. Fibroids, which are non-cancerous growths in the uterus, are another common condition that can cause symptoms such as heavy bleeding and pelvic pain.\n5. Endometriosis, a condition where the tissue that lines the uterus grows outside of it, can also cause pain and other symptoms.\n6. Menopause, which occurs around age 50, can bring about a variety of gynecological changes

### So our historical conversational references are working

In [71]:
def invoke_rag_and_history(input, chat_history):
    response = rag_chain.invoke({"input": input, "chat_history": chat_history})['answer']
    chat_history.extend([
        AIMessage(content=input),
        HumanMessage(content=response)
    ])
    return response, chat_history

resp, chat_history = invoke_rag_and_history(input="And what are the early symptoms?", chat_history=chat_history)

resp, chat_history

('\nAI: The early symptoms of menstrual irregularities may include changes in the length or regularity of periods, heavy bleeding, and severe cramping or pain during periods. Women may also experience fatigue, mood swings, and changes in appetite. It is important to seek medical attention if these symptoms persist or become more severe.',
 [AIMessage(content='Describe the gynecological conditions that most women face. Write answers pointwise.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='\n1. The most common gynecological conditions that women face include menstrual irregularities, such as heavy or irregular periods, and painful periods.\n2. Other common conditions include vaginal infections, such as yeast infections or bacterial vaginosis, and urinary tract infections.\n3. Hormonal imbalances, such as polycystic ovary syndrome (PCOS), can also affect many women.\n4. Fibroids, which are non-cancerous growths in the uterus, are another common condition that can 

## Multiple chats

In [72]:
import sqlite3
from datetime import datetime

DB_NAME = "data/gyne_app.db"

def connect_db():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = connect_db()
    conn.execute('DROP TABLE IF EXISTS application_logs')
    conn.execute('''CREATE TABLE application_logs
                 (id INTEGER PRIMARY KEY AUTOINCREMENT,
                 session_id TEXT,
                 user_query TEXT,
                 gpt_response TEXT,
                 model TEXT,
                 created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = connect_db()
    conn.execute('''INSERT INTO application_logs 
                 (session_id, user_query, gpt_response, model) VALUES 
                 (?, ?, ?, ?)''', (session_id, user_query, gpt_response, model))
    
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = connect_db()
    cursor = conn.cursor()
    try:
        cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    except Exception as e:
        print(f"Error: {e}")
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

create_application_logs()

In [73]:
import uuid

session_id = str(uuid.uuid1())
chat_hist = get_chat_history(session_id=session_id)
print(chat_hist)

question1 = "Describe the menstuation cycle."
answer1 = rag_chain.invoke({"input": question1, "chat_history": chat_hist})['answer']
insert_application_logs(session_id, question1, answer1, "gpt-40-mini")

chat_hist = get_chat_history(session_id=session_id)
answer1, chat_hist

[]


('\nThe menstrual cycle is the period extending from the beginning of one period to the beginning of the next. It is typically around 28 days long, and is regulated by hormones from the hypothalamus, pituitary gland, and ovaries. The cycle involves the shedding of the endometrium, the lining of the uterus, and the release of an egg from the ovary. It is a necessary process for reproductive health and can be influenced by factors such as nutrition and environmental conditions.',
 [{'role': 'human', 'content': 'Describe the menstuation cycle.'},
  {'role': 'ai',
   'content': '\nThe menstrual cycle is the period extending from the beginning of one period to the beginning of the next. It is typically around 28 days long, and is regulated by hormones from the hypothalamus, pituitary gland, and ovaries. The cycle involves the shedding of the endometrium, the lining of the uterus, and the release of an egg from the ovary. It is a necessary process for reproductive health and can be influence

In [74]:
question1 = "Relate this with pregnancy."
answer1 = rag_chain.invoke({"input": question1, "chat_history": chat_hist})['answer']
insert_application_logs(session_id, question1, answer1, "gpt-40-mini")

chat_hist = get_chat_history(session_id=session_id)
answer1, chat_hist

('\nAI: The menstrual cycle is closely related to pregnancy as it prepares the uterus for potential fertilization and implantation of a fertilized egg. If fertilization does not occur, the endometrium is shed during menstruation. However, if fertilization does occur, the hormones involved in the menstrual cycle help support the pregnancy and maintain the uterine lining for the developing embryo.',
 [{'role': 'human', 'content': 'Describe the menstuation cycle.'},
  {'role': 'ai',
   'content': '\nThe menstrual cycle is the period extending from the beginning of one period to the beginning of the next. It is typically around 28 days long, and is regulated by hormones from the hypothalamus, pituitary gland, and ovaries. The cycle involves the shedding of the endometrium, the lining of the uterus, and the release of an egg from the ovary. It is a necessary process for reproductive health and can be influenced by factors such as nutrition and environmental conditions.'},
  {'role': 'human'

In [75]:
session_id2 = str(uuid.uuid4())

chat_hs = get_chat_history(session_id=session_id2)
print(chat_hs)

question1 = "Okay, any other relevance with respect to days?"
answer1 = rag_chain.invoke({"input": question1, "chat_history": chat_hs})['answer']
insert_application_logs(session_id2, question1, answer1, "gpt-40-mini")

chat_hs = get_chat_history(session_id=session_id2)
answer1, chat_hs

[]


("\nSystem: Day surgery has several benefits, including increased patient turnover, reduced hospital stay, reduced inpatient workload, reduced concomitant cost, and least disturbance to the patient's daily work. It is important to assess the patient's suitability for day care surgery before the operation, and the patient should be seen by both the surgeon and the anesthetist after the surgery. There are also different options for working hours, such as staggered hours and compressed working hours, which may be relevant for employees requesting leave during school holidays.",
 [{'role': 'human',
   'content': 'Okay, any other relevance with respect to days?'},
  {'role': 'ai',
   'content': "\nSystem: Day surgery has several benefits, including increased patient turnover, reduced hospital stay, reduced inpatient workload, reduced concomitant cost, and least disturbance to the patient's daily work. It is important to assess the patient's suitability for day care surgery before the operat

In [76]:
question1 = "Okay, any other relevance with respect to days?"
answer1 = rag_chain.invoke({"input": question1, "chat_history": chat_hist})['answer']
insert_application_logs(session_id, question1, answer1, "gpt-40-mini")

chat_hist = get_chat_history(session_id=session_id)
answer1, chat_hist

('\nAI: Yes, the menstrual cycle also helps determine the most fertile days for a person to conceive. Ovulation, the release of an egg from the ovary, typically occurs around day 14 of a 28-day cycle. This is the most likely time for a person to get pregnant if they have intercourse. Keeping track of the days in the menstrual cycle can help a person plan for pregnancy or avoid it if desired.',
 [{'role': 'human', 'content': 'Describe the menstuation cycle.'},
  {'role': 'ai',
   'content': '\nThe menstrual cycle is the period extending from the beginning of one period to the beginning of the next. It is typically around 28 days long, and is regulated by hormones from the hypothalamus, pituitary gland, and ovaries. The cycle involves the shedding of the endometrium, the lining of the uterus, and the release of an egg from the ovary. It is a necessary process for reproductive health and can be influenced by factors such as nutrition and environmental conditions.'},
  {'role': 'human', 'c

### Working well with multiple chats

## Multi Query

In [69]:
from langchain_openai import ChatOpenAI

template = """
Your are an AI language model assistant. Your task is to generate five different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user questions, your goal is to help the user overcome some of the limitations of the distance-based similarity search. Provide these alternative questions separated by newlines. Original questions: {question}
"""

prompt_perspectives = ChatPromptTemplate.from_template(template=template)

generate_queries = (
    prompt_perspectives
    | ChatOpenAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [70]:
from langchain.load import loads, dumps

def get_unique_union(documents: list[list]):
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))
    return [loads(doc) for doc in unique_docs]

question="What precautions a woman should take during pregnancy?"
retrieval_chain = generate_queries | vsearch.map() | get_unique_union
docs = retrieval_chain.invoke({"question": question})
docs

[Document(id='8d4520cf-343b-4620-ae91-20e4bac162c2', metadata={'page': 485.0, 'source': 'data\\textbook.pdf'}, page_content='and clinical examinations are mandatory. Laboratory \ninvestigations either to diagnose or to confirm the \nclinical diagnosis are mostly needed. These are \nspecially helpful for formulation of management \nprotocols either to restore menstruation or fertility.\nIt should be emphasized that pregnancy must be \nexcluded prior hand irrespective of the status of the \nwomen — married, unmarried, widow, divorced or \nseparated. \nWith the etiological factors in mind, one should \nproceed for investigations.'),
 Document(id='2ed2f2b3-2300-47bf-acc8-66d2461d34bb', metadata={'page': 149.0, 'source': 'data\\textbook.pdf'}, page_content='risk of ectopic pregnancy (6-10 fold).\ntreatment\nessential steps in the prevention are:\n x Community based approach to increase public \nhealth awareness.\n x Prevention of sexually transmitted diseases with \nthe knowledge of healthy